In [463]:
import json
import os

import requests
from openai import OpenAI

In [464]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1", api_key=os.getenv("GROQ_API_KEY")
)

In [465]:
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    print(data["current"])
    return data["current"]

In [466]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather data like temperature, wind speed for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [467]:
system_prompt = (
    "You are a helpful weather assistant who nicely answers in full sentences."
)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the temperature like in Kolkata today?"},
]

In [468]:
completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [469]:
# completion.model_dump()

In [470]:
completion.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1t0t', function=Function(arguments='{"latitude": 22.5782, "longitude": 88.3639}', name='get_weather'), type='function')]))

In [471]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)


def checkforToolCalls(completion):
    if completion.choices[0].message.tool_calls:
        print("Tool call found")
        for tool_call in completion.choices[0].message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            messages.append(completion.choices[0].message)
            result = call_function(name, args)
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result),
                }
            )

In [472]:
checkforToolCalls(completion)

Tool call found
{'time': '2025-02-10T14:30', 'interval': 900, 'temperature_2m': 23.5, 'wind_speed_10m': 5.8}


In [473]:
messages

[{'role': 'system',
  'content': 'You are a helpful weather assistant who nicely answers in full sentences.'},
 {'role': 'user', 'content': "What's the temperature like in Kolkata today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1t0t', function=Function(arguments='{"latitude": 22.5782, "longitude": 88.3639}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_1t0t',
  'content': '{"time": "2025-02-10T14:30", "interval": 900, "temperature_2m": 23.5, "wind_speed_10m": 5.8}'}]

In [474]:
completion_2 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [475]:
final_response = completion_2.choices[0].message.content
final_response

"It's 23.5 degrees Celsius in Kolkata today, and the wind speed is around 5.8 meters per second."

In [480]:
messages.append(completion_2.choices[0].message)

In [481]:
messages

[{'role': 'system',
  'content': 'You are a helpful weather assistant who nicely answers in full sentences.'},
 {'role': 'user', 'content': "What's the temperature like in Kolkata today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1t0t', function=Function(arguments='{"latitude": 22.5782, "longitude": 88.3639}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_1t0t',
  'content': '{"time": "2025-02-10T14:30", "interval": 900, "temperature_2m": 23.5, "wind_speed_10m": 5.8}'},
 ChatCompletionMessage(content="It's 23.5 degrees Celsius in Kolkata today, and the wind speed is around 5.8 meters per second.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]

In [482]:
messages.append(
    {"role": "user", "content": "What is the temperature of Kolkata in Fahrenheit?"}
)

In [483]:
completion_3 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [484]:
completion_3.choices[0].message.content

"To convert the temperature from Celsius to Fahrenheit, we'll use the formula: (°C × 9/5) + 32 = °F. \n\nUsing this formula with the temperature in Kolkata (23.5 degrees Celsius), we get:\n(23.5 × 9/5) + 32 = 74.3 degrees Fahrenheit.\n\nSo, the temperature of Kolkata is 74.3 degrees Fahrenheit."